Query Operators - Comparison

1. How many documents in the sample_training.zips collection have fewer than 1000 people listed in the pop field?



      db.zips.find({pop:{"$lt":1000}}).count()


2. What is the difference between the number of people born in 1998 and the
   number of people born after 1998 in the sample_training.trips collection?

      db.trips.find({"birth year": {"$eq": 1998}}).count();
      db.trips.find({"birth year": {"$gt": 1998}}).count();

3. Using the sample_training.routes collection find out which of the
   following statements will return all routes that have at least one stop
   in them?

      db.routes.find({ "stops": { "$gt": 0 }}).pretty()

Query Operators - Logic

1. How many businesses in the sample_training.inspections dataset have the
   inspection result "Out of Business" and belong to the Home Improvement
   Contractor - 100 sector?

      db.inspections.find( {result: "Out of Business", sector: "Home Improvement Contractor - 100"} );


2. How many zips in the sample_training.zips dataset are neither over-
   populated nor under-populated? In this case, we consider population over 1,000,000 to be over-populated
   and under 5,000 to be under-populated.

      db.zips.find({ "pop": { "$gte": 5000, "$lte": 1000000 }}).count()

3. How many companies in the sample_training.companies dataset were either
   founded in 2004, or in the month of October and either have the social
   category_code or web category_code?

       db.companies.find({ "$and": [
                            { "$or": [ { "founded_year": 2004 },
                                    { "founded_month": 10 } ] },
                            { "$or": [ { "category_code": "web" },
                                    { "category_code": "social" }]}]}).count()


Expressive Query Operator

How many companies in the sample_training.companies collection have the same
permalink as their twitter_username?

      db.companies.find({ "$expr": { "$eq": [ "$permalink", "$twitter_username"] }}).count()
   
Array Operators

1. What is the name of the listing in the sample_airbnb.listingsAndReviews dataset accommodate more than 6 people and has exactly 50 reviews?

      db.listingsAndReviews.find({"accommodates": {"$gt": 6},"number_of_reviews":50}).count()

2. How many documents have the property_type House, and include Changing table as one of the amenities?

      db.listingsAndReviews.find({"property_type": "House", "amenities": {"$all":["Changing table"]} }).count()

Array Operators and Projection

How many companies in the sample_training.companies collection have offices
in the city of Seattle?


      db.companies.find({"offices.city":"Seattle"})

Array Operators and Sub-Documents

1. Latitude decreases in value as you move west.

How many trips in the sample_training.trips collection started at stations that are to the west of the -74 latitude coordinate?

      db.trips.find( { "start station location.coordinates.0": { "$lt": -74 } } )

1. How many inspections from the sample_training.inspections collection were
   conducted in the city of NEW YORK?

      db.inspections.find( { "address.city": "NEW YORK" } ).count()


Aggregation Framework

1. Using the aggregation framework find all documents that have Wifi as one
   of the amenities. Only include price and address in the resulting cursor.

      db.listingsAndReviews.aggregate([
                                       { "$match": { "amenities": "Wifi" } },
                                       { "$project": { "price": 1,
                                                      "address": 1,
                                                      "_id": 0 }}]).pretty()


2. Which countries have listings in the sample_airbnb database?

      db.listingsAndReviews.aggregate([ { "$project": { "address": 1, "_id": 0 }},
                                       { "$group": { "_id": "$address.country" }}])

3. How many countries have listings in the sample_airbnb database?

      db.listingsAndReviews.aggregate([
                                       { "$project": { "address": 1, "_id": 0 }},
                                       { "$group": { "_id": "$address.country",
                                                      "count": { "$sum": 1 } } }
                                    ])

sort() and limit()

1. Find the least populated ZIP code in the zips collection.

    db.zips.find({ "pop": { "$ne": 0 }},{ "zip": 1, "pop": 1 }).sort({ "pop": 1 }).limit(1)

2. Find the most populated ZIP code in the zips collection.

   db.zips.find({ "pop": { "$ne": 0 }},{ "zip": 1, "pop": 1 }).sort({ "pop": -1 }).limit(1)

3. Find the top ten most populated ZIP codes.
   db.zips.find({ "pop": { "$ne": 0 }},{ "zip": 1, "pop": 1 }).sort({ "pop": -1 }).limit(10)

4. Get results sorted in increasing order by population, and decreasing
   order by city name.

   db.zips.find().sort({"pop":1, "name":-1})

Introduction to Indexes

Create two separate indxes to support the following queries:

db.trips.find({"birth year": 1989})

   db.trips.createIndex({ "birth year": 1989 })

   ## Compound index 

db.trips.find({"start station id": 476}).sort("birth year": 1)

   db.trips.createIndex({ "start station id": 1, "birth year": 1 })


Inserting New Documents - insert() and errors:

1. Get a random document from a collection

   use sample_analytics
   db.sample_analytics.findOne()
   
2. Copy this random document, and insert it back to the collection. Do you get
   a "Duplicate Key" error?

   db.accounts.findOne();

   db.accounts.insert({
  _id: ObjectId("5ca4bbc7a2dd94ee581623a5"),
  account_id: 433811,
  limit: 10000,
  products: [
    'CurrencyService',
    'InvestmentFund',
    'Brokerage',
    'InvestmentStock'
  ]
})

3. Insert that document into the collection without the _id field to get a
   successfull insert. Did it work?

   db.accounts.insert({
  account_id: 433811,
  limit: 10000,
  products: [
    'CurrencyService',
    'InvestmentFund',
    'Brokerage',
    'InvestmentStock'
  ]
})

Inserting New Documents - insert() order:

1. Insert three test documents into the inspections collection
   

   use sample_training;
   
   db.inspections.insert([{"test":1},{"test":2},{"test":3}])


2. Insert the same three documents into the inspections collection. Did it
   work? Why?

Yes, it did work. It worked because the _id field was not specified. 

3. Insert these three test documents into the inspections collection. Did it
   work? Why?


   db.inspections.insert([{"_id": 1, "test": 1},{"_id": 1, "test": 2},{"_id": 3,"test": 3}])

   No, it didn't work. This is because there's more than one document the same _id (specifically, _id as 1, 2, and 3).


4. Try the same insert as above but make it unordered.

   db.inspections.insert([{"_id": 1, "test": 1},{"_id": 1, "test": 2},{"_id": 3,"test": 3},{"ordered":false}])


5. Try this command. Did it work? Why?
   
   db.inspection.insert([{ "_id":1, "test": 1 },{ "_id": 3,"test": 3 }])

It worked because this is a different collection name- it's creating new records under the name "inspection" not "inspections."


Updating Documents  - mongo shell

1. Find all documents in the zips collection where the zip field is equal to
   12434.

      db.zips.find({"zip" : "12434"}).pretty();
   
2. Find all documents in the zips collection where the city field is equal to
   "HUDSON".

   db.zips.find({"city" : "HUDSON"}).pretty();


3. Find how many documents in the zips collection have the city field is equal
   to "HUDSON".

   db.zips.find({"city" : "HUDSON"}).count()

   16

4. Update all documents in the zips collection where the city field is equal
   to "HUDSON" by adding 10 to the current value of the "pop" field.

   db.zips.updateMany({ "city": "HUDSON" }, { "$inc": { "pop": 10 } })
   
5. Update a single document in the zips collection where the zip field is
   equal to 12534 by setting the value of the "pop" field to 17630.

   db.zips.updateOne({ "zip": 12534 }, { "$set": { "pop": 17630 } });


6. Update a single document in the zips collection where the zip field is
   equal to 12534 by setting the value of the "population" field to 17630.

   db.zips.updateOne({ "zip": "12534" }, { "$set": { "population": 17630 } });

7. Find all documents in the grades collection where the student_id is 151,
   and the class_id field is 339.

   db.grades.find({"student_id" : 151,"class_id":339});

8. Find all documents in the grades collection where the student_id is 250,
   and the class_id field is 339.
   
   db.grades.find({"student_id" : 250,"class_id":339});

9. Update one document in the grades collection where the student_id is 250,
   and the class_id field is 339, by adding a document element to the "scores"
   array.

   

Practice Question:

People often confuse NEW YORK City as the capital of New York state, when in
reality the capital of New York state is ALBANY.

Add a boolean field "capital?" to all documents pertaining to ALBANY NY, and
NEW YORK, NY. The value of the field should be true for all ALBANY documents
and false for all NEW YORK documents.


Deleting Documents and Collections

1. Look at all the documents in the inspections collection that have test field
   equal to 1.

   db.inspections.find({"test":1});

2. Look at all the documents in the inspections collection that have test field
   equal to 3.

   db.inspections.find({"test":3});

3. Delete all the documents from the inspections collection that have test
   field equal to 1

   db.inspections.deleteMany({ "test": 1 })

4. Delete one document from the inspections collection that has test field
   equal to 3

   db.inspections.deleteOne({ "test": 3 })

5. Inspect what is left of the inspection collection.


6. View what collections are present in the sample_training database.


7. Drop the inspection collection

   db.inspection.drop()